In [1]:
#!/usr/bin/env python3
"""
identify.py

This script reads the training graphs and labels, performs frequent subgraph mining,
selects the most discriminative subgraphs (up to 100), and saves them for later use.
"""

import pickle
import networkx as nx
import gspan_mining.gspan as gspan

In [2]:
def parse_graphs(file_path):
    graphs = []
    current_graph = nx.Graph()
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):  # New graph indicator
                if current_graph.number_of_nodes() > 0:
                    graphs.append(current_graph)
                    current_graph = nx.Graph()
                continue
            if line.startswith('v'):
                # Format: v node_id label
                parts = line.split()
                node_id, label = parts[1], parts[2]
                current_graph.add_node(int(node_id), label=label)
            elif line.startswith('e'):
                # Format: e node1 node2 label
                parts = line.split()
                u, v, label = int(parts[1]), int(parts[2]), parts[3]
                current_graph.add_edge(u, v, label=label)
        # Append last graph if exists
        if current_graph.number_of_nodes() > 0:
            graphs.append(current_graph)
    return graphs


def gspan_mining(file_path, min_support=2, min_num_vertices=1):
    with open('tmp_graph.txt', 'w') as outfile:
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('#'):  # New graph indicator
                    outfile.write('\nt # \n')
                else:
                    outfile.write(line + '\n')

    # Run gSpan
    gs = gspan.gSpan('tmp_graph.txt', min_support,  min_num_vertices)
    gs.run()

    return gs._frequent_subgraphs


def convert_gspan_to_nx(gspan_graph : gspan.Graph):
    graph = nx.Graph()
    gspan_graph = gspan_graph.to_graph()
    for node in gspan_graph.vertices:
        graph.add_node(gspan_graph.vertices[node].vid, label=gspan_graph.vertices[node].vlb)
    for frm in gspan_graph.vertices:
        edges = gspan_graph.vertices[frm].edges
        for to in edges:
            graph.add_edge(frm, to, label=edges[to].elb)
    return graph


def select_discriminative_subgraphs(candidate_subgraphs, graphs, labels, max_features=100):
    discriminative_subgraphs = []
    for subgraph in candidate_subgraphs:
        support_positive = 0
        support_negative = 0
        for i, graph in enumerate(graphs):
            is_iso = nx.isomorphism.GraphMatcher(subgraph, graph).subgraph_is_isomorphic()
            if labels[i] == 1 and is_iso:
                support_positive += 1
            elif labels[i] == 0 and is_iso:
                support_negative += 1
        if support_positive > 0 and support_negative > 0:
            discriminative_subgraphs.append((subgraph, support_positive, support_negative))
    discriminative_subgraphs.sort(key=lambda x: abs(x[1] - x[2]), reverse=True)
    return [x[0] for x in discriminative_subgraphs[:max_features]]


def load_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                labels.append(int(line))
    return labels


In [3]:
GRAPHS = './q3_datasets/NCI-H23/graphs.txt'
LABELS = './q3_datasets/NCI-H23/labels.txt'

In [4]:
print("Parsing training graphs...")
graphs = parse_graphs(GRAPHS)
labels = load_labels(LABELS)

Parsing training graphs...


In [5]:
print("Mining candidate subgraphs using gSpan...")
candidates = gspan_mining(GRAPHS, min_support=0.1*len(labels), min_num_vertices=2)
print(f"Found {len(candidates)} candidate subgraphs.")

Mining candidate subgraphs using gSpan...
t # 0
v 0 1
v 1 2
e 0 1 0

Support: 32149

-----------------

t # 1
v 0 1
v 1 2
v 2 2
e 0 1 0
e 1 2 0

Support: 31077

-----------------

t # 2
v 0 1
v 1 2
v 2 2
v 3 2
e 0 1 0
e 1 2 0
e 2 3 1

Support: 24137

-----------------

t # 3
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
e 0 1 0
e 1 2 0
e 2 3 1
e 3 4 0

Support: 22410

-----------------

t # 4
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
v 5 2
e 0 1 0
e 1 2 0
e 2 3 1
e 3 4 0
e 4 5 1

Support: 21328

-----------------

t # 5
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
v 5 2
v 6 2
e 0 1 0
e 1 2 0
e 2 3 1
e 3 4 0
e 4 5 1
e 5 6 0

Support: 20903

-----------------

t # 6
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
v 5 2
v 6 2
e 0 1 0
e 1 2 0
e 1 6 1
e 2 3 1
e 3 4 0
e 4 5 1
e 5 6 0

Support: 15699

-----------------

t # 7
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
v 5 2
v 6 2
v 7 2
e 0 1 0
e 0 7 0
e 1 2 0
e 1 6 1
e 2 3 1
e 3 4 0
e 4 5 1
e 5 6 0

Support: 11895

-----------------

t # 8
v 0 1
v 1 2
v 2 2
v 3 2
v 4 2
v 5 2
v 6 2
v 7 2
v 8 2
e 0 1 0
e 0 7 0
e 1 2 

In [6]:
nx_candidates = [convert_gspan_to_nx(candidate) for candidate in candidates]

In [7]:
print("Selecting discriminative subgraphs...")
discriminative_subgraphs = select_discriminative_subgraphs(nx_candidates, graphs, labels, max_features=100)
print(f"Selected {len(discriminative_subgraphs)} discriminative subgraphs.")

Selecting discriminative subgraphs...
Selected 100 discriminative subgraphs.


In [11]:
print("Saving discriminative subgraphs...")
with open('subgraphs.pkl', 'wb') as f:
    pickle.dump(discriminative_subgraphs, f)
print(f"Discriminative subgraphs saved to 'subgraphs.pkl'.")

Saving discriminative subgraphs...
Discriminative subgraphs saved to 'subgraphs.pkl'.


In [ ]:
# rename tmp_graph.txt to tmp_graph.txt.bak
import os
os.rename('tmp_graph.txt', 'tmp_graph.txt.bak')
